In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
#Qn1. Import data
path = r'C:\Users\Hiroshi\Documents\12-2023 Instacart Basket Analysis'
ords_prods_merge_updated = pd.read_pickle(os.path.join(path, '02 Data', '02 Prepared Data', 'ords_prods_merge_updated.pkl'))

In [3]:
#Create subset
df = ords_prods_merge_updated[:1000000]

In [4]:
ords_prods_merge_updated.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_2days,busiest_period_of_day
0,2539329,1,1,2,8,NaN,1,196,1,0,Soda,77.0,7.0,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders
1,2539329,1,1,2,8,NaN,1,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91.0,16.0,12.5,both,Mid-range product,Regularly busy,Regularly busy,Average orders
2,2539329,1,1,2,8,NaN,1,12427,3,0,Original Beef Jerky,23.0,19.0,4.4,both,Low-range product,Regularly busy,Regularly busy,Average orders
3,2539329,1,1,2,8,NaN,1,26088,4,0,Aged White Cheddar Popcorn,23.0,19.0,4.7,both,Low-range product,Regularly busy,Regularly busy,Average orders
4,2539329,1,1,2,8,NaN,1,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54.0,17.0,1.0,both,Low-range product,Regularly busy,Regularly busy,Average orders


In [6]:
#Aggregate average number of orders per department for subset
df.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1.0,14.792915
2.0,17.091743
3.0,17.919589
4.0,17.891973
5.0,15.214270
6.0,15.382228
7.0,17.699986
8.0,16.485269


In [7]:
#Qn4. Create a flag according to maximum number of order for each customer.
#Step1. Create new column for maximum order of each customer
ords_prods_merge_updated['max_order'] = ords_prods_merge_updated.groupby(['user_id'])['order_number'].transform(np.max)

C:\Users\Hiroshi\AppData\Local\Temp\ipykernel_8752\644489870.py:3: FutureWarning: The provided callable <function amax at 0x0000017527A36C00> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge_updated['max_order'] = ords_prods_merge_updated.groupby(['user_id'])['order_number'].transform(np.max)


In [9]:
#Qn4. Create a flag according to maximum number of order for each customer
#Step2. Create loyalty flag
ords_prods_merge_updated.loc[ords_prods_merge_updated['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prods_merge_updated.loc[(ords_prods_merge_updated['max_order'] <= 40) & (ords_prods_merge_updated['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
ords_prods_merge_updated.loc[ords_prods_merge_updated['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

C:\Users\Hiroshi\AppData\Local\Temp\ipykernel_8752\2175303983.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Loyal customer' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ords_prods_merge_updated.loc[ords_prods_merge_updated['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'


In [10]:
ords_prods_merge_updated['loyalty_flag'].value_counts(dropna=False)

loyalty_flag
Regular customer    15891507
Loyal customer      10294027
New customer         6249525
Name: count, dtype: int64

In [12]:
# Qn2. Find aggregate mean for the order_number column grouped by departement_id
ords_prods_merge_updated.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1.0,15.457838
2.0,17.277920
3.0,17.170395
4.0,17.811403
5.0,15.215751
6.0,16.439806
7.0,17.225802
8.0,15.340650


Qn3. The result for the subset is within difference of 1 compared to the result of the entire dataframe.

In [13]:
#Qn5. Aggregate the mean, min and max of prices of products purchased by loyal, regular and new customers.
ords_prods_merge_updated.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

Price range of product purchased by regular and new customers are the same but loyal customers have a lower price mean than regular or new customers. 

In [14]:
#Qn6
#Step1. Create new column for the mean of the prices of products purchased by each customer
ords_prods_merge_updated['mean_price'] = ords_prods_merge_updated.groupby(['user_id'])['prices'].transform(np.mean)

C:\Users\Hiroshi\AppData\Local\Temp\ipykernel_8752\3568853091.py:3: FutureWarning: The provided callable <function amin at 0x0000017527A36DE0> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  ords_prods_merge_updated['mean_price'] = ords_prods_merge_updated.groupby(['user_id'])['prices'].transform(np.min)


In [16]:
#Qn6. 
#Step2. Create the spending flag
ords_prods_merge_updated.loc[ords_prods_merge_updated['mean_price'] < 10, 'spending_flag'] = 'Low spender'
ords_prods_merge_updated.loc[(ords_prods_merge_updated['mean_price'] >= 10), 'spending_flag'] = 'High spender'

C:\Users\Hiroshi\AppData\Local\Temp\ipykernel_8752\4036538445.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Low spender' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ords_prods_merge_updated.loc[ords_prods_merge_updated['mean_price'] < 10, 'spending_flag'] = 'Low spender'


In [19]:
ords_prods_merge_updated['spending_flag'].value_counts(dropna=False)

spending_flag
Low spender     32433516
High spender        1543
Name: count, dtype: int64

In [20]:
#Qn7
#Step1. Create new column for the median of the days_since_prior_order for each customer
ords_prods_merge_updated['median_days_since_prior_order'] = ords_prods_merge_updated.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

C:\Users\Hiroshi\AppData\Local\Temp\ipykernel_8752\1945791516.py:3: FutureWarning: The provided callable <function median at 0x0000017527B74FE0> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge_updated['median_days_since_prior_order'] = ords_prods_merge_updated.groupby(['user_id'])['days_since_prior_order'].transform(np.median)


In [22]:
#Qn7. 
#Step2. Create the order frequency flag
ords_prods_merge_updated.loc[ords_prods_merge_updated['median_days_since_prior_order'] > 20, 'order_freq_flag'] = 'Non-frequent customer'
ords_prods_merge_updated.loc[(ords_prods_merge_updated['median_days_since_prior_order'] > 10) & (ords_prods_merge_updated['median_days_since_prior_order'] <= 20), 'order_freq_flag'] = 'Regular customer'
ords_prods_merge_updated.loc[ords_prods_merge_updated['median_days_since_prior_order'] <= 10, 'order_freq_flag'] = 'Frequent customer'

C:\Users\Hiroshi\AppData\Local\Temp\ipykernel_8752\2253269273.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Non-frequent customer' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ords_prods_merge_updated.loc[ords_prods_merge_updated['median_days_since_prior_order'] > 20, 'order_freq_flag'] = 'Non-frequent customer'


In [23]:
ords_prods_merge_updated['order_freq_flag'].value_counts(dropna=False)

order_freq_flag
Frequent customer        21578273
Regular customer          7217066
Non-frequent customer     3639720
Name: count, dtype: int64

In [24]:
# Qn9. Export data in pickle format.
ords_prods_merge_updated.to_pickle(os.path.join(path, '02 Data','02 Prepared Data', 'ords_prods_merge_updated2.pkl'))